In [ ]:
import psycopg2
import pandas as pd
pd.options.display.max_columns = 40

### Import raw dataset

#### Challenge - Build code to collect data directly from hockey-reference.com

In [ ]:
## load DF for dates
df = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/in/njd_2023.csv")
abv_team_names = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/in/team_abv.csv")

team_abv = 'NJD'
season = '2023'
reg_or_playoffs = 'R'

In [ ]:
df = df.loc[:, :"Opponent.4"]
df.drop(columns=["Unnamed: 8","Unnamed: 14"],inplace=True)
df.rename(columns={'Unnamed: 0':'GP', 'Unnamed: 1':"Date", 'Unnamed: 2':"Home", 'Unnamed: 3':"Opp_Name", 'Unnamed: 4':"GF",
       'Unnamed: 5':"GA", 'Unnamed: 6':"Result", 'Unnamed: 7':"OT", 'Team':"Team_S", 'Team.1':"Team_PIM", 'Team.2':"Team_PPG",
       'Team.3':"Team_PPO", 'Team.4':"Team_SHG", 'Opponent':"Opp_S", 'Opponent.1':"Opp_PIM", 'Opponent.2':"Opp_PPG",
       'Opponent.3':"Opp_PPO", 'Opponent.4':"Opp_SHG"},inplace=True)
df.drop(0,inplace=True)
df.Home = [0 if x == "@" else 1 for x in df.Home]
df.OT = [1 if x == "OT" else 0 for x in df.OT]
df['Team_Abv'] = team_abv
df['Season'] = reg_or_playoffs
df['Season_Year'] = season
df = df.merge(abv_team_names, left_on=['Team_Abv'],right_on=["ABV"], how='left')

In [ ]:
df.rename(columns={'ID':"team_id"},inplace=True)
df.drop(columns='ABV',inplace=True)
df["Key"] = df["team_id"].astype(str) + "-" + season + "-" + reg_or_playoffs + df["GP"].astype(str)
df['Date'] = df.Date.astype(str)
df['Date'] = df.Date.str.replace('-','')
df['Date'] = df.Date.astype(int)
df = df[['Key','team_id','Season_Year','GP','Season','Date','Home','Opp_Name','GF','GA','Result','OT','Team_S','Team_PIM', 
    'Team_PPG', 'Team_PPO', 'Team_SHG', 'Opp_S', 'Opp_PIM', 'Opp_PPG', 'Opp_PPO', 'Opp_SHG']]
df.head()

#### Connect to database

In [ ]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=hockey_yt_project user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)

#### Find last game in database

In [ ]:
try: 
    cur.execute("SELECT max(date) FROM fact_gamelog where team_id = 18;")

except psycopg2.Error as e:
    print("Select Error")
    print(e)

row = cur.fetchone()
while row:
    last_game_in_db = row
    row = cur.fetchone()

In [ ]:
last_game_in_db[0]

#### Delete last row - as an example

In [ ]:
try: 
    cur.execute(f"DELETE FROM fact_gamelog WHERE date = {last_game_in_db[0]};")
except psycopg2.Error as e:
    print("Delete Error")
    print(e)

#### Query new last game

In [ ]:
try: 
    cur.execute("SELECT max(date) FROM fact_gamelog where team_id = 18;")

except psycopg2.Error as e:
    print("Select Error")
    print(e)

row = cur.fetchone()
while row:
    last_game_in_db = row
    row = cur.fetchone()

print(last_game_in_db[0])

#### Filter for rows after last game in db

In [ ]:
missing_games = df[df.Date > last_game_in_db[0]]
missing_games

In [ ]:
gamelog_insert = ("""INSERT INTO fact_gamelog(
                  key,
                  team_id,
                  season_year,
                  gp,
                  season,
                  date,
                  home,
                  opp_name,
                  gf,
                  ga,
                  result,
                  ot,
                  team_s,
                  team_pim,
                  team_ppg,
                  team_ppo,
                  team_shg,
                  opp_s,
                  opp_pim,
                  opp_ppg,
                  opp_ppo,
                  opp_shg)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
               """)

#### Insert into DB

In [ ]:
try:
    for i, row in missing_games.iterrows():
        #print(list(row))
        cur.execute(gamelog_insert, list(row))

except psycopg2.Error as e:
    print("Error: could not insert rows")
    print(e)